In [86]:
import pandas as pd
import numpy as np

In [87]:
df = pd.read_csv('AllTraffic-UK-2023.csv', skiprows=9)
df = df[df['Country'] == 'United Kingdom']
df.drop(['Country'], axis=1, inplace=True)
df.dropna(inplace=True)
df['Landing page'] = df['Landing page'].astype(str)

df_organic = pd.read_csv('OrganicTraffic-UK-2023.csv', skiprows=9)
df_organic = df_organic[df_organic['Country'] == 'United Kingdom']
df_organic.drop(['Country', 'Ad revenue'], axis=1, inplace=True)
df_organic.rename(columns={'Landing page + query string': 'Landing page'}, inplace=True)
df['Landing page'] = df['Landing page'].astype(str)


df = pd.merge(df, df_organic, on='Landing page', how='outer')
df.dropna(inplace=True)
df = df[df['Key events_x'] != 0]
df = df[df['Key events_y'] != 0]
df['Sessions/KeyEvents All'] = df['Sessions'] / df['Key events_x']
df['Sessions/KeyEvents Organic'] = df['Engaged sessions'] / df['Key events_y']
df.rename(columns={'Engagement rate_x': 'Engagement rate all', 'Engagement rate_y': 'Engagement rate organic'}, inplace=True)

In [88]:
temp = df[['Landing page', 'Sessions', 'Sessions/KeyEvents All', 'Sessions/KeyEvents Organic', 'Engagement rate all', 'Engagement rate organic', 'Average engagement time per session']]
temp.sort_values(by='Sessions', ascending=False).reset_index(drop=True).round(2).to_csv('Session-KeyEvents 2023 Analysis.csv', index=False)

In [37]:
import matplotlib
list_colours = []
for name, hex in matplotlib.colors.cnames.items():
    name = name.title()
    list_colours.append(name)

# list_colours = []
colours_df = pd.read_csv('colours.csv')
colours_df['colours'] = colours_df['colours'].str.strip()
for j in colours_df['colours'].unique().tolist():
    list_colours.append(j)

for i in list_colours:
    i = i.title()

list_colours = list(dict.fromkeys(list_colours))

df['Colour'] = df['Landing page'].str.findall(r'(?i)\b(?:{})\b'.format('|'.join(list_colours))).apply(', '.join)

df2 = df['Colour'].str.split(',', expand=True)
df['Colour'] = df2[0]

In [38]:
df['PageType'] = df['Landing page'].str.split('-').str[-1]
df['fPageType'] = df['PageType'].astype(str).str[0]
df['fPageType'].value_counts()

fPageType
p    3350
c     129
t     119
m      70
/      19
j       7
i       6
s       4
b       2
(       1
l       1
h       1
Name: count, dtype: int64

In [39]:
cat_df = df[df['fPageType'] == 'c']
prod_df = df[df['fPageType'] == 'p']
other_cat_df = df[(df['fPageType'] != 'p') & (df['fPageType'] != 'c')]
other_cat_df['PageType'] = other_cat_df['Landing page'].str.split('-').str[-2]
other_cat_df['fPageType'] = other_cat_df['PageType'].astype(str).str[0]
other_cat_df = other_cat_df[other_cat_df['fPageType'] == 'c']
cat_df = pd.concat([cat_df, other_cat_df], ignore_index=True)

C:\Users\retai\AppData\Local\Temp\ipykernel_8136\2404332529.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  other_cat_df['PageType'] = other_cat_df['Landing page'].str.split('-').str[-2]
C:\Users\retai\AppData\Local\Temp\ipykernel_8136\2404332529.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  other_cat_df['fPageType'] = other_cat_df['PageType'].astype(str).str[0]


In [50]:
# list(prod_df[(prod_df['Landing page'].str.contains('mens-c4/leather-jackets-c25/')) & (prod_df['Colour'] == 'tan')]['Landing page'])

In [41]:
cat_df.describe()

,Sessions,Users,New users,Engagement rate,Average engagement time per session,Key events,Total revenue
count,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000
mean,936.109023,847.966165,715.699248,0.675336,68.923753,7.958647,735.702405
std,3107.988450,2806.570415,2460.930004,0.103842,32.018383,22.056578,2280.724491
min,36.000000,33.000000,2.000000,0.365132,10.047619,0.000000,0.000000
25%,81.000000,75.000000,61.000000,0.621657,46.016528,0.000000,0.000000
50%,180.000000,163.500000,127.000000,0.691478,65.807408,1.000000,92.970000
75%,562.500000,524.750000,428.250000,0.743325,86.541951,5.000000,510.025000
max,31487.000000,27967.000000,24439.000000,1.000000,177.957447,181.000000,22218.684256


In [42]:
prod_df.describe()

,Sessions,Users,New users,Engagement rate,Average engagement time per session,Key events,Total revenue
count,3350.000000,3350.000000,3350.000000,3350.000000,3350.000000,3350.000000,3350.000000
mean,195.549254,169.628955,136.060597,0.564091,51.795563,2.181791,162.880218
std,384.804852,342.351528,303.650336,0.116957,31.801840,5.010538,375.917568
min,36.000000,2.000000,0.000000,0.068182,0.545455,0.000000,-1462.170000
25%,54.000000,45.000000,30.000000,0.486486,33.477564,0.000000,0.000000
50%,88.000000,74.000000,52.000000,0.578125,46.973341,1.000000,34.980000
75%,178.000000,153.750000,118.000000,0.653770,63.129900,2.000000,199.000000
max,7269.000000,6407.000000,5943.000000,0.861111,1072.164384,103.000000,7377.590000


In [43]:
temp = cat_df.copy()
temp['S/K'] = temp['Sessions'] / temp['Key events']
temp = temp[temp['Key events'] != 0]
temp = temp[temp['Key events'] > 30]
temp.sort_values(by='S/K', ascending=True)[:20]

,Landing page,Sessions,Users,New users,Engagement rate,Average engagement time per session,Key events,Total revenue,Colour,PageType,fPageType,S/K
130,/sheepskin-fur-c5/sheepskin-footwear-c46/fenla...,1971,1804,1633,0.737697,67.762050,61,2663.540413,,c46/fenland,c,32.311475
18,/luggage-c7/wash-toiletry-bags-c55,2739,2582,2329,0.787879,78.424608,77,3547.520000,,c55,c,35.571429
33,/gift-accessories-c8/ladies-gloves-c38,1123,1025,837,0.677649,70.519145,31,1298.400000,,c38,c,36.225806
11,/gift-accessories-c8/mens-gloves-c37,3865,3580,3298,0.775162,68.364812,81,2916.500000,,c37,c,47.716049
26,/black-friday-c164,1667,1469,1052,0.709658,125.799640,31,3965.850000,black,c164,c,53.774194
24,/gift-accessories-c8/purses-c36,1830,1645,1093,0.644809,94.853005,34,1384.400000,,c36,c,53.823529
15,/ladies-c3,3357,2948,1959,0.563003,122.228180,55,7300.560000,,c3,c,61.036364
19,/gift-accessories-c8/wallets-c16,2388,2237,1977,0.748325,86.453518,32,1192.408158,,c16,c,74.625000
12,/mens-c4,3848,3191,2122,0.662162,137.753378,50,5820.300000,,c4,c,76.960000
7,/luggage-c7/holdall-c15,6830,6219,5489,0.720059,88.849341,85,11137.718840,,c15,c,80.352941


In [44]:
list(temp.sort_values(by='S/K', ascending=True)[:20]['Landing page'])

['/sheepskin-fur-c5/sheepskin-footwear-c46/fenland-m10',
 '/luggage-c7/wash-toiletry-bags-c55',
 '/gift-accessories-c8/ladies-gloves-c38',
 '/gift-accessories-c8/mens-gloves-c37',
 '/black-friday-c164',
 '/gift-accessories-c8/purses-c36',
 '/ladies-c3',
 '/gift-accessories-c8/wallets-c16',
 '/mens-c4',
 '/luggage-c7/holdall-c15',
 '/handbags-c6/cross-body-bag-c87',
 '/handbags-c6/ashwood-c30',
 '/mens-c4/man-bags-briefcases-c12',
 '/ladies-c3/deals-for-her-c161',
 '/luggage-c7',
 '/ladies-c3/leather-jackets-c9',
 '/handbags-c6/backpacks-c89',
 '/handbags-c6',
 '/mens-c4/leather-jackets-c25']